In [5]:
import json

def get_tickets():
    with open('support_tickets.json', 'r') as file:
        data = json.load(file)
    return data

def get_ticket(index):
    with open('support_tickets.json', 'r') as file:
        data = json.load(file)
    return data['tickets'][index]

def stringify_ticket(ticket):
    return f"Title: {ticket['title']}\nDescription: {ticket['description']}"

ticket = get_ticket(1)
print(stringify_ticket(ticket))


Title: Request to Block Cases from Specific Region
Description: I need to stop receiving cases from the APAC region as I don't have the language capabilities to handle them effectively. Is there a way to update my assignment rules to exclude these cases?


In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
import markdown


def load_knowledge_base():
    with open('knowledge_wiki.md', 'r') as file:
        kb_content = file.read()
    
    chunks = [chunk.strip() for chunk in kb_content.split('###') if chunk.strip()]
    
    clean_chunks = []
    for chunk in chunks:
        html = markdown.markdown(chunk)
        clean_text = html.replace('<strong>', '').replace('</strong>', '') \
                        .replace('<p>', '').replace('</p>', '') \
                        .replace('<h3>', '').replace('</h3>', '') \
                        .replace('<ol>', '').replace('</ol>', '') \
                        .replace('<li>', '').replace('</li>', '')
        clean_chunks.append(clean_text)
    
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = FAISS.from_texts(
        clean_chunks,
        embeddings,
        metadatas=[{"chunk_id": i, "source": "knowledge_wiki"} for i in range(len(clean_chunks))]
    )
    return vectorstore


def find_relevant_knowledge(ticket_string, vectorstore, top_k=3):
    relevant_chunks = vectorstore.similarity_search(ticket_string, k=top_k)
    return [doc.page_content for doc in relevant_chunks]

In [7]:
vectorstore = load_knowledge_base()

/Users/ayaanrahim/Documents/ResolveIQ/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
ticket = get_ticket(2)
ticket_string = stringify_ticket(ticket)
print("Ticket: ", ticket_string)
relevant_knowledge = find_relevant_knowledge(ticket_string, vectorstore)
print("\nRelevant knowledge base sections:")
for i, chunk in enumerate(relevant_knowledge, 1):
    print(f"\nChunk {i}:")
    print(chunk)

Ticket:  Title: Time Zone Update Required
Description: I recently relocated to the EST time zone, but my working hours are still showing in PST. Need assistance updating my profile to reflect the correct time zone so my availability is properly tracked.

Relevant knowledge base sections:

Chunk 1:
Updating Time Zone Settings
To update your working hours and reflect the correct time zone:
1. Go to Profile Settings: Locate the "Time Zone" section.
2. Select the Correct Time Zone: Choose your current time zone from the dropdown.
3. Verify Your Schedule: Ensure that your work hours align with the new time zone.
4. Save Changes: Refresh the system and test by checking availability.

Chunk 2:
Troubleshooting Auto Assignment Issues
If you are not receiving auto assignments despite being marked as "Available," follow these steps:
1. Verify System Status: Check for ongoing outages or maintenance in the assignment system.
2. Check Queue Configuration: Ensure your queue is properly configured and

In [10]:
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage, SystemMessage

chat_model = ChatOllama(model="llama3.2:1b")

def generate_response(ticket_string, relevant_knowledge):
    messages = [
        SystemMessage(content="You are a helpful AI assistant. Use the relevant knowledge to solve the ticket."),
        HumanMessage(content="Ticket: " + ticket_string),
        HumanMessage(content="Relevant knowledge: " + relevant_knowledge)
    ]
    response = chat_model.invoke(messages)
    return response.content

response = generate_response(ticket_string, relevant_knowledge[0])
print(response)

I'd be happy to help you update your working hours and reflect the correct time zone in your profile.

To proceed, I'll guide you through the steps:

1. Go to Profile Settings
2. Locate the "Time Zone" section
3. Select the Correct Time Zone
4. Verify Your Schedule
5. Save Changes

Please confirm that you've already relocated to the EST (Eastern Standard Time) time zone and your working hours are still set to PST (Pacific Standard Time).


In [ ]:
def solve_ticket(ticket_index):
    vectorstore = load_knowledge_base()
    
    ticket = get_ticket(ticket_index)
    ticket_string = stringify_ticket(ticket)

    relevant_knowledge = find_relevant_knowledge(ticket_string, vectorstore)
    response = generate_response(ticket_string, relevant_knowledge[0])

    return response

print(solve_ticket(0))

Based on the ticket description, I'll provide step-by-step troubleshooting to help resolve the auto assignment issue.

**Step 1: Verify System Status**
Check the system status to ensure there are no ongoing outages or maintenance in the assignment system. You can try checking the system's status page or contacting the system administrator to confirm that everything is operating smoothly.

**Step 2: Check Queue Configuration**
Verify that your queue is properly configured and there are no conflicting filters. Make sure that:

* Your queue is enabled and not set to "Hidden"
* There are no duplicate or conflicting queues
* The filter criteria (e.g., job type, location) are correct

**Step 3: Manually Refresh Status**
 Toggle your availability off and on again to refresh your status in the system. This will help you check if the issue is related to a temporary system glitch.

**Additional Steps**

* If none of the above steps resolve the issue, it's possible that there's a technical issue 

In [4]:
import sqlite3
import json

class EmployeeDatabase:

    def __init__(self, db_path="employee_support.db"):
        self.db_path = db_path
        self.setup_database()
        self.populate_table()

    def get_connection(self):
        return sqlite3.connect(self.db_path)
    
    def setup_database(self):
        with self.get_connection() as conn:
            cursor = conn.cursor()
            cursor.execute("DROP TABLE IF EXISTS employees")
            cursor.execute("""
                           CREATE TABLE IF NOT EXISTS employees (
                            alias TEXT PRIMARY KEY,
                            timezone TEXT,
                            country TEXT,
                            support_region TEXT,
                            skills TEXT,
                            line_of_business TEXT,
                            manager TEXT,
                            email TEXT,
                            language TEXT,
                            status TEXT,
                            product TEXT,
                            name TEXT)
                           """)
            conn.commit()
    
    def populate_table(self):
        with self.get_connection() as conn:
            cursor = conn.cursor()
            sample_employees = [
        {
            "alias": "jsmith",
            "timezone": "EST",
            "country": "United States",
            "support_region": "Americas",
            "skills": json.dumps(["Troubleshooting", "Technical Writing", "Cloud Infrastructure"]),
            "line_of_business": "Enterprise",
            "manager": "mwilson",
            "email": "john.smith@company.com",
            "language": json.dumps(["English", "Spanish"]),
            "status": "Active",
            "product": "Cloud Platform",
            "name": "John Smith"
        },
        {
            "alias": "agarcia",
            "timezone": "PST",
            "country": "United States",
            "support_region": "Americas",
            "skills": json.dumps(["Database Management", "Performance Tuning", "Security"]),
            "line_of_business": "SMB",
            "manager": "mwilson",
            "email": "ana.garcia@company.com",
            "language": json.dumps(["English", "Spanish", "Portuguese"]),
            "status": "Active",
            "product": "Database Solutions",
            "name": "Ana Garcia"
        },
        {
            "alias": "tkumar",
            "timezone": "IST",
            "country": "India",
            "support_region": "APAC",
            "skills": json.dumps(["API Integration", "Mobile Development", "Backend Systems"]),
            "line_of_business": "Enterprise",
            "manager": "rpatel",
            "email": "tej.kumar@company.com",
            "language": json.dumps(["English", "Hindi", "Telugu"]),
            "status": "Active",
            "product": "API Gateway",
            "name": "Tej Kumar"
        },
        {
            "alias": "lwang",
            "timezone": "CST",
            "country": "China",
            "support_region": "APAC",
            "skills": json.dumps(["Machine Learning", "Data Analytics", "Cloud Architecture"]),
            "line_of_business": "Enterprise",
            "manager": "yzhan",
            "email": "li.wang@company.com",
            "language": json.dumps(["English", "Mandarin", "Cantonese"]),
            "status": "Active",
            "product": "ML Platform",
            "name": "Li Wang"
        },
        {
            "alias": "mmueller",
            "timezone": "CET",
            "country": "Germany",
            "support_region": "EMEA",
            "skills": json.dumps(["Security", "Compliance", "Network Infrastructure"]),
            "line_of_business": "Enterprise",
            "manager": "kschmidt",
            "email": "max.mueller@company.com",
            "language": json.dumps(["English", "German", "French"]),
            "status": "Active",
            "product": "Security Suite",
            "name": "Max Mueller"
        },
        {
            "alias": "slee",
            "timezone": "KST",
            "country": "South Korea",
            "support_region": "APAC",
            "skills": json.dumps(["Mobile Development", "Frontend Development", "UX Design"]),
            "line_of_business": "SMB",
            "manager": "jkim",
            "email": "sun.lee@company.com",
            "language": json.dumps(["English", "Korean", "Japanese"]),
            "status": "Training",
            "product": "Mobile SDK",
            "name": "Sun Lee"
        },
        {
            "alias": "olivia",
            "timezone": "GMT",
            "country": "United Kingdom",
            "support_region": "EMEA",
            "skills": json.dumps(["Product Management", "Technical Support", "Customer Success"]),
            "line_of_business": "Enterprise",
            "manager": "robert",
            "email": "olivia.brown@company.com",
            "language": json.dumps(["English", "French"]),
            "status": "Active",
            "product": "Enterprise Suite",
            "name": "Olivia Brown"
        },
        {
            "alias": "rpatel",
            "timezone": "IST",
            "country": "India",
            "support_region": "APAC",
            "skills": json.dumps(["Team Leadership", "Strategy", "Technical Architecture"]),
            "line_of_business": "Enterprise",
            "manager": "sarah",
            "email": "raj.patel@company.com",
            "language": json.dumps(["English", "Hindi", "Gujarati"]),
            "status": "Active",
            "product": "All Products",
            "name": "Raj Patel"
        }
    ]
            cursor.executemany("""
                               INSERT INTO employees (alias, timezone, country, support_region, skills, line_of_business, manager, email, language, status, product, name)
                               VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                               """, [(emp["alias"], emp["timezone"], emp["country"], emp["support_region"],
                                    emp["skills"], emp["line_of_business"], emp["manager"], emp["email"],
                                    emp["language"], emp["status"], emp["product"], emp["name"]) 
                                    for emp in sample_employees])
            conn.commit()
            
    
    def is_table_empty(self):
        with self.get_connection() as conn:
            cursor = conn.cursor()
            cursor.execute("SELECT COUNT(*) FROM employees")
            count = cursor.fetchone()[0]
            return count == 0
        
    def execute_query(self, query):
        with self.get_connection() as conn:
            cursor = conn.cursor()
            cursor.execute(query)
            output = cursor.fetchall()
            return output
        

In [5]:
db = EmployeeDatabase()

In [6]:
results = db.execute_query("SELECT * FROM employees")

for row in results:
    print(row)

('jsmith', 'EST', 'United States', 'Americas', '["Troubleshooting", "Technical Writing", "Cloud Infrastructure"]', 'Enterprise', 'mwilson', 'john.smith@company.com', '["English", "Spanish"]', 'Active', 'Cloud Platform', 'John Smith')
('agarcia', 'PST', 'United States', 'Americas', '["Database Management", "Performance Tuning", "Security"]', 'SMB', 'mwilson', 'ana.garcia@company.com', '["English", "Spanish", "Portuguese"]', 'Active', 'Database Solutions', 'Ana Garcia')
('tkumar', 'IST', 'India', 'APAC', '["API Integration", "Mobile Development", "Backend Systems"]', 'Enterprise', 'rpatel', 'tej.kumar@company.com', '["English", "Hindi", "Telugu"]', 'Active', 'API Gateway', 'Tej Kumar')
('lwang', 'CST', 'China', 'APAC', '["Machine Learning", "Data Analytics", "Cloud Architecture"]', 'Enterprise', 'yzhan', 'li.wang@company.com', '["English", "Mandarin", "Cantonese"]', 'Active', 'ML Platform', 'Li Wang')
('mmueller', 'CET', 'Germany', 'EMEA', '["Security", "Compliance", "Network Infrastruct